
# ДЗ 3.
С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом,чтобы в качестве строк были категории продуктов, в качестве столбцов были дни,а значениями были средние значения выручки от продаж за соответствующий день

In [30]:
# Решение:

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, window
from pyspark.sql.types import DateType

# Инициализация SparkSession
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

# Создание DataFrame с предоставленными данными
data = [
   ("2023-11-20", "Electronics", 100, 12000),
   ("2023-11-21", "Electronics", 110, 13000),
   ("2023-11-22", "Electronics", 105, 12500),
   ("2023-11-20", "Clothing", 300, 15000),
   ("2023-11-21", "Clothing", 280, 14000),
   ("2023-11-22", "Clothing", 320, 16000),
   ("2023-11-20", "Books", 150, 9000),
   ("2023-11-21", "Books", 200, 12000),
   ("2023-11-22", "Books", 180, 10000)
]
columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, columns)

# Преобразование столбца "date" в тип Date
df = df.withColumn("date", col("date").cast(DateType()))

# Расчет средней выручки от продаж для каждой категории продукта
window = Window.partitionBy("category")
df_avg_revenue = df.withColumn("avg_revenue", F.avg("revenue").over(window))
df_avg_revenue.show()
# Применение операции pivot для преобразования данных
pivot_df = df_avg_revenue.groupBy("category").pivot("date").agg(avg("revenue"))

# # Вывод результата
pivot_df.show()

# # Завершение работы SparkSession

spark.stop()


+----------+-----------+--------+-------+------------------+
|      date|   category|quantity|revenue|       avg_revenue|
+----------+-----------+--------+-------+------------------+
|2023-11-20|      Books|     150|   9000|10333.333333333334|
|2023-11-21|      Books|     200|  12000|10333.333333333334|
|2023-11-22|      Books|     180|  10000|10333.333333333334|
|2023-11-20|   Clothing|     300|  15000|           15000.0|
|2023-11-21|   Clothing|     280|  14000|           15000.0|
|2023-11-22|   Clothing|     320|  16000|           15000.0|
|2023-11-20|Electronics|     100|  12000|           12500.0|
|2023-11-21|Electronics|     110|  13000|           12500.0|
|2023-11-22|Electronics|     105|  12500|           12500.0|
+----------+-----------+--------+-------+------------------+

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15